# Mounting Azure Data Lake Storage Gen 2 

This Notebook will allow the users to create a mount point to ADLS Gen 2 account by using the widgets attached to the notebook

# Overview
To create mount point,
1. Select the widgets for the Storage Account Name and Container Name
2. Enter a file path if you want to mount to a different path in the container
3. Select the Secret Key Associated with the Storage Account in the widget

# Setting up the widgets

This part of the code uses the creates the widgets at the top of the notebook.

**Storage Account Name**: Is the name of the storage account you want to link to <br>
<br>
**Container Name**: Is the name of the container within the storage account you want to set up the mount point
<br>
<br>
**File Path**: this is a free text field. If there is a partcular directory within in the container that you would like to connect a mount point to.
<br>
<br>
**Secret Scope**: This is a provisioned Scope within the Databricks backed Secret scope.
<br>
<br>
**Secret Key:** Is the Actual Key stored in Databricks Secret Scope associated with the Storage Account. One Secret Key will allow you to build multiple mount points within that storage account, irrespective of the container.


In [0]:
dbutils.widgets.dropdown(name='storage_account_name',defaultValue='ingest00landingsbox',choices=['ingest00landingsbox','ingest00curatedsbox','ingest00rawsbox','ingest00externalsbox'],label='Storage Account Name')
dbutils.widgets.dropdown(name='container_name',defaultValue='landing',choices=['landing','bronze','silver','gold','html-template','raw','external-csv'],label='Container Name')
dbutils.widgets.text(name='file_path',defaultValue='SQLServer/Sales/IRIS/dbo/',label='File Path')
dbutils.widgets.dropdown(name='secret_scope',defaultValue='dbScope',choices=['dbScope','ExternalStorageKey'],label=' Secret scope')
dbutils.widgets.dropdown(name='secret_key',defaultValue='storageKey',choices=['storageKey','CuratedStorageSBOXKey','RawStorageSBOX','externalStorageKey'],label=' Secret Key')

# Collect the user defined settings

This part of the code uses the get function to collect in input parameters required to create the mount point.

In [0]:
storage_account_name = dbutils.widgets.get('storage_account_name')
container_name = dbutils.widgets.get('container_name')
file_path = dbutils.widgets.get('file_path')
mountPoint = "/mnt/"+container_name
print(mountPoint)
storage_account_key = dbutils.secrets.get('dbScope', 'storageKey')
print(file_path)


In [0]:
# mountPoint_new = mountPoint.replace(mountPoint.split('/')[2][:8], '', 1)
# display(mountPoint)
# display(mountPoint_new)

In [0]:
display(storage_account_name[8:])

In [0]:
dbutils.secrets.listScopes()

In [0]:
if any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  dbutils.fs.unmount(mountPoint)
try:
    dbutils.fs.mount(
        source= "wasbs://{}@{}.blob.core.windows.net/".format(container_name, storage_account_name,file_path),
        mount_point=mountPoint,
        extra_configs={'fs.azure.account.key.' +storage_account_name+'.blob.core.windows.net':dbutils.secrets.get(dbutils.widgets.get('secret_scope'), dbutils.widgets.get('secret_key'))}
    )
    print(f"Successfully mounted {mountPoint}")
except Exception as e:
    print(f"Error mounting {mountPoint} : {e}")